In [1]:
import pickle
import numpy as np
# Load MNIST data from mnist.pkl
with open('mnist.pkl', 'rb') as f:
    mnist_data = pickle.load(f, encoding='latin1')

# Extract training, validation, and test sets
train_data, valid_data, test_data = mnist_data

# Unpack the data into inputs and labels
X_train, Y_train = train_data
X_val, Y_val = valid_data
X_test, Y_test = test_data

# Convert inputs to numpy arrays
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

# Convert labels to numpy arrays
Y_train = np.array(Y_train)
Y_val = np.array(Y_val)
Y_test = np.array(Y_test)